In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
from Loader import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import datetime

#38.3 s
#8:55 m

In [2]:
BATCH_SIZE = 128

# x_train = None
# x_val = None
x_train = DataLoader(BATCH_SIZE, 'training_generator')
x_val = DataLoader(BATCH_SIZE, 'training_generator', val=True)

In [14]:
lossHistory = []
valHistory = []

class NnueCallbacks(tf.keras.callbacks.Callback):
  def on_batch_end(self, batch, logs=None):
    return
    lossHistory[-1].append(logs['loss'])
    if batch and batch % 15000 == 0:

      pred = self.model.predict(x_val)
      loss = (pred.flatten() - x_val.labels.flatten()[0:len(x_val)*BATCH_SIZE]) ** 2
      loss = np.sum(loss) / len(loss)
      valHistory[-1].append(loss)

      for i in lossHistory:
        plt.plot(i[20:], linewidth=0.5)
        plt.savefig("loss.png", dpi=400)
      plt.clf()

      for i in valHistory:
        plt.plot(i, linewidth=0.5)
        plt.savefig("val.png", dpi=400)
      
      plt.clf()

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [15]:
from keras import backend as K
def clipped_relu(x):
    return K.relu(x, max_value=4)

input = Input(shape=(769,))
layers = Dense(128, activation=clipped_relu)(input)
layers = Dense(32, activation=clipped_relu)(layers)
layers = Dense(32, activation=clipped_relu)(layers)
layers = Concatenate()([layers, input])
layers = Dense(1, activation='sigmoid')(layers)

model = Model(inputs=[input], outputs=[layers])

# model = Sequential()
# model.add(Dense(128, input_shape=(769,), activation=clipped_relu))
# model.add(Dense(32, activation=clipped_relu))
# model.add(Dense(32, activation=clipped_relu))
# model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [ ]:
model = tf.keras.models.load_model("production/")

In [16]:
lossHistory.append([])
valHistory.append([])
model.fit(x_train, validation_data=x_val, epochs=100, callbacks=[NnueCallbacks(), EarlyStopping(patience=1)])

# Epoch 1/100
# 7812/7812 [==============================] - 41s 5ms/step - loss: 0.0180 - mae: 0.0885 - val_loss: 0.0247 - val_mae: 0.1000
# Epoch 2/100
# 7812/7812 [==============================] - 38s 5ms/step - loss: 0.0124 - mae: 0.0724 - val_loss: 0.0245 - val_mae: 0.0994
# Epoch 3/100
# 7812/7812 [==============================] - 39s 5ms/step - loss: 0.0109 - mae: 0.0677 - val_loss: 0.0236 - val_mae: 0.0945
# Epoch 4/100
# 7812/7812 [==============================] - 39s 5ms/step - loss: 0.0099 - mae: 0.0646 - val_loss: 0.0230 - val_mae: 0.0929
# Epoch 5/100
# 7812/7812 [==============================] - 38s 5ms/step - loss: 0.0092 - mae: 0.0623 - val_loss: 0.0237 - val_mae: 0.0957

Epoch 1/100
7812/7812 [==============================] - 41s 5ms/step - loss: 0.0179 - mae: 0.0884 - val_loss: 0.0245 - val_mae: 0.1008
Epoch 2/100
7812/7812 [==============================] - 39s 5ms/step - loss: 0.0124 - mae: 0.0726 - val_loss: 0.0239 - val_mae: 0.0960
Epoch 3/100
7812/7812 [==============================] - 40s 5ms/step - loss: 0.0109 - mae: 0.0678 - val_loss: 0.0232 - val_mae: 0.0966
Epoch 4/100
7812/7812 [==============================] - 40s 5ms/step - loss: 0.0099 - mae: 0.0647 - val_loss: 0.0230 - val_mae: 0.0937
Epoch 5/100
7812/7812 [==============================] - 40s 5ms/step - loss: 0.0093 - mae: 0.0627 - val_loss: 0.0228 - val_mae: 0.0924
Epoch 6/100
7812/7812 [==============================] - 39s 5ms/step - loss: 0.0088 - mae: 0.0610 - val_loss: 0.0227 - val_mae: 0.0913
Epoch 7/100
7812/7812 [==============================] - 41s 5ms/step - loss: 0.0085 - mae: 0.0599 - val_loss: 0.0226 - val_mae: 0.0907
Epoch 8/100
7812/7812 [=========================

In [7]:
import Features
import numpy as np

FEN = "r3n1k1/2qbbppp/4p3/3p2P1/2p2P2/2N1P3/Pr1B2BP/R2Q1RK1 b - - 2 19"
features = Features.get(FEN)
model.predict(np.array([features]))

array([[0.15982483]], dtype=float32)

In [17]:
model.save("production")

2022-10-06 09:32:21.659838: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: production/assets
